In [1]:
import pandas as pd
import pickle
from annoy import AnnoyIndex

with open('./models/generate/relations/dataset.pickle', 'rb') as file:
    relations_df = pickle.load(file)

with open('./models/generate/relations/model.pickle', 'rb') as file:
    relations_model = pickle.load(file)

with open('./models/generate/relations/csr.pickle', 'rb') as file:
    relations_csr = pickle.load(file)

with open('./models/generate/relations/item_to_id.pickle', 'rb') as file:
    item_to_id = pickle.load(file)
 
with open('./models/generate/relations/user_to_id.pickle', 'rb') as file:
    user_to_id = pickle.load(file)

with open('./models/generate/content/dataset.pickle', 'rb') as file:
    content_df = pickle.load(file)

annoy = AnnoyIndex(1024, 'angular')
annoy.load('./models/generate/content/index.ann')

True

In [2]:
content_df

,id,titlte,description,screen,coordinates,embeddings
0,5e7c05fe9134ed0019ddbd47,Ресторан «The Сад»,Ресторан «The Сад» славится своей субтропическ...,restaurant,NaN,"[tensor(0.0127), tensor(-0.2021), tensor(-0.63..."
1,5f9a85226ccc9d00185d40b1,"Речная прогулка от Флотилии «Рэдиссон Ройал», ...",Флотилия «Рэдиссон Ройал» — это десять речных ...,event,"{'lng': 37.5971937943344, 'lat': 55.7298828224...","[tensor(0.6727), tensor(-0.2268), tensor(0.075..."
2,624c5b848ee45eeeba8665cd,"Солнце, море и нарзан: путешествие из Сочи в К...","Пожалуй, нет двух более непохожих друг на друг...",track,NaN,"[tensor(0.3980), tensor(-0.5097), tensor(-0.97..."
3,5e5d676d88e8040019cda132,Колоннада Исаакиевского собора,Колоннада Исаакиевского собора стала не только...,event,"{'lng': 30.306485, 'lat': 59.933905}","[tensor(0.3120), tensor(-0.5846), tensor(-0.62..."
4,61ba021114277dec6442cbb7,"Инстатур «Музыка волн, музыка ветра»",Это фотопутешествие познакомит с Петербургом б...,excursion,NaN,"[tensor(0.2111), tensor(-0.3817), tensor(0.033..."
...,...,...,...,...,...,...
21632,64e6064e4cd8f478ca980011,Мастер-класс по изготовлению интерьерного укра...,Лев и русалка - любимые образы у мастеров наро...,excursion,NaN,"[tensor(1.0509), tensor(-0.4288), tensor(-0.79..."
21633,64e609d34cd8f478ca984c81,Мастер-класс «Вещь! Предметный разговор!»,В начале ХХ века прорывные идеи русского аванг...,excursion,NaN,"[tensor(0.9174), tensor(-0.3028), tensor(-0.91..."
21634,63ca53e74ddfad493ca2f584,She,"SHE — это проект будущего, который концентриру...",restaurant,NaN,"[tensor(0.5407), tensor(-0.2792), tensor(-0.18..."
21635,64e634039760fe4b09ef6d36,Театральная игра «Что может быть свободнее дет...,Участники в игровой форме учатся креативным ме...,excursion,NaN,"[tensor(1.5860), tensor(0.0432), tensor(0.1347..."


In [3]:
relations_model.recommend(1, relations_csr[1])

(array([20048, 19183, 17447, 20956, 20553, 17484, 17557, 17554, 17525,
        19439], dtype=int32),
 array([0.019685  , 0.01580156, 0.01290512, 0.01061905, 0.00909937,
        0.00769095, 0.0075808 , 0.00628322, 0.00608956, 0.00594939],
       dtype=float32))

In [4]:
for i in range(1000):
    relations_model.recommend(i, relations_csr[i], N=200000)

In [5]:
positives = []

for user in user_to_id.values():
    predictions, weights = relations_model.recommend(user, relations_csr[user])
    for i in range(len(predictions)):
        base_ann = predictions[0]
        if i == 0:
            base_ann = predictions[1]
        positives.append({
            'als_weight': weights[i],
            'user': user,
            'item': predictions[i],
            'score': 1,
            'type': 'ALS',
            'ann_weight': annoy.get_distance(predictions[i], base_ann),
        })

In [88]:
for user in user_to_id.values():
    if user % 100 == 0: print(user)
    item_ids = relations_df[relations_df.client_id==user]['external_Id'].tolist()
    user_scores, user_weights = relations_model.recommend(user, relations_csr[user], N=relations_csr.shape[1])
    for item_id in item_ids:
        predicted_ids = annoy.get_nns_by_item(item_id, 5)
        for predicted_id in predicted_ids:
            positives.append({
                'ann_weight': annoy.get_distance(item_id, predicted_id),
                'als_weight': user_weights[user_scores.tolist().index(predicted_id)],
                'user': user,
                'item': predicted_id,
                'score': 1,
                'type': 'ANN'
            })

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [35]:
len(positives)

495405

In [7]:
positives_dict = dict()
used = set()
for i, positive in enumerate(positives):
    if i % 1000 == 0:
        print(i)
    key = f'{positive["user"]}_{positive["item"]}'
    if key not in used:
        positives_dict.update({key: positive})
    used.add(key)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [8]:
type(relations_model.recommend(0, relations_csr[0], N=relations_csr.shape[1])[1][-1])

numpy.float32

In [89]:
from random import shuffle, randint
negatives = []

for user in user_to_id.values():
    if user % 100 == 0:
        print(user)
    user_scores, user_weights = relations_model.recommend(user, relations_csr[user], N=relations_csr.shape[1])
    base_item = user_scores[0]
    pairs = [[user_scores[i], user_weights[i]] for i in range(len(user_scores))]
    pairs = list(
        filter(
            lambda x: f'{user}_{x[0]}' not in used and x[1] != 0.0, pairs
        )
    )
    cands = [pairs[randint(10, len(pairs)-1)] for i in range(20)]
    # shuffle(pairs)
    for pair in cands:
        negatives.append({
            'als_weight': pair[1],
            'user': user,
            'item': pair[0],
            'score': 0,
            'ann_weight': annoy.get_distance(pair[0], base_item),
            'type': 'ALS'
        })

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [91]:
len(negatives)

619480

In [92]:
factors = relations_model.user_factors

In [93]:
ranking_data = positives + negatives

In [94]:
ranking_df = pd.DataFrame(ranking_data)

In [95]:
user_to_embeddings = dict()

for record in content_df[['id', 'embeddings']].to_dict(orient='records'):
    user_to_embeddings.update({item_to_id[record['id']]: record['embeddings']})

In [96]:
ranking_df['user_embeddings'] = ranking_df['user'].map(lambda x: factors[x])

In [97]:
ranking_df['item_embeddings'] = ranking_df['item'].map(lambda x: user_to_embeddings[x])

In [98]:
ranking_df

,als_weight,user,item,score,type,ann_weight,user_embeddings,item_embeddings
0,0.029088,0,15881,1,ALS,1.046396,"[0.032258548, 0.016014647, 0.043277506, 0.0033...","[tensor(0.9950), tensor(-0.3174), tensor(0.349..."
1,0.028906,0,17447,1,ALS,1.046396,"[0.032258548, 0.016014647, 0.043277506, 0.0033...","[tensor(1.1045), tensor(-0.2708), tensor(-0.19..."
2,0.022216,0,19438,1,ALS,0.965781,"[0.032258548, 0.016014647, 0.043277506, 0.0033...","[tensor(1.1287), tensor(-0.4808), tensor(-0.21..."
3,0.020531,0,20966,1,ALS,0.993904,"[0.032258548, 0.016014647, 0.043277506, 0.0033...","[tensor(0.6971), tensor(-0.1542), tensor(-0.50..."
4,0.019209,0,17467,1,ALS,1.146236,"[0.032258548, 0.016014647, 0.043277506, 0.0033...","[tensor(0.3799), tensor(0.1506), tensor(0.1648..."
...,...,...,...,...,...,...,...,...
1149670,-0.001308,30973,17494,0,ALS,1.133042,"[-0.02112704, 0.023279285, 0.028742164, 0.0420...","[tensor(1.1219), tensor(-0.3860), tensor(0.236..."
1149671,0.002348,30973,17497,0,ALS,1.133042,"[-0.02112704, 0.023279285, 0.028742164, 0.0420...","[tensor(1.1219), tensor(-0.3860), tensor(0.236..."
1149672,0.000002,30973,1723,0,ALS,1.027679,"[-0.02112704, 0.023279285, 0.028742164, 0.0420...","[tensor(-0.0379), tensor(-0.3869), tensor(-0.2..."
1149673,-0.000018,30973,20050,0,ALS,1.147000,"[-0.02112704, 0.023279285, 0.028742164, 0.0420...","[tensor(1.0104), tensor(0.1180), tensor(-0.948..."


In [85]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.2/26.2 MB 29.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 33.7 MB/s eta 0:00:0000:0100:01
  Using cached tenacity-8.2.3-py3-none-any.whl (24 kB)


In [99]:
from catboost import CatBoostRanker, Pool, MetricVisualizer
from sklearn.utils import shuffle

ranking_df = shuffle(ranking_df)

In [19]:
ranking_df

,als_weight,user,item,score,type,ann_weight,user_embeddings,item_embeddings
233170,0.000254,23317,1370,1,ALS,0.801739,"[0.04855662, 0.046632905, 0.056401394, 0.00484...","[tensor(0.0790), tensor(-0.3859), tensor(0.248..."
185033,0.000222,18503,0,1,ALS,0.981332,"[0.048556622, 0.046632923, 0.056401405, 0.0048...","[tensor(0.0127), tensor(-0.2021), tensor(-0.63..."
37187,0.004702,3718,17456,1,ALS,0.939195,"[0.010926505, 0.013073395, -0.02074563, -0.059...","[tensor(0.7246), tensor(-0.3824), tensor(-0.62..."
248935,0.000864,24893,1995,1,ALS,0.929970,"[-0.005791197, 0.021445552, -0.0072684987, 0.0...","[tensor(0.7156), tensor(-0.7176), tensor(-0.24..."
66453,0.000222,6645,0,1,ALS,0.981332,"[0.048573613, 0.04663743, 0.056408666, 0.00483...","[tensor(0.0127), tensor(-0.2021), tensor(-0.63..."
...,...,...,...,...,...,...,...,...
306040,0.002251,30604,19181,1,ALS,1.041433,"[-0.001424451, 0.0068824645, 0.030187076, -0.0...","[tensor(0.3822), tensor(-0.4778), tensor(-0.55..."
295583,0.000222,29558,0,1,ALS,0.981332,"[0.048556633, 0.046632893, 0.056401383, 0.0048...","[tensor(0.0127), tensor(-0.2021), tensor(-0.63..."
308233,0.010352,30823,19183,1,ALS,0.958182,"[-0.045148496, 0.057825677, 0.046114385, -0.09...","[tensor(0.7114), tensor(-0.5806), tensor(-0.38..."
173237,0.002307,17323,5270,1,ALS,1.000105,"[-0.0084899515, 0.017118795, 0.009117012, -0.0...","[tensor(0.5714), tensor(-0.3149), tensor(-0.73..."


In [100]:
ranking_df = ranking_df.rename(columns={'score': 'y'})
ranking_df = ranking_df.rename(columns={'y': 'label'})

In [101]:
ranking_df

,als_weight,user,item,label,type,ann_weight,user_embeddings,item_embeddings
703899,0.000200,8685,4801,0,ALS,1.077025,"[-0.0014244509, 0.0068824575, 0.030187078, -0....","[tensor(0.7962), tensor(-0.1597), tensor(-0.97..."
1130692,0.000016,30024,19924,0,ALS,1.100282,"[-0.00142445, 0.0068824664, 0.030187085, -0.02...","[tensor(0.6092), tensor(0.1876), tensor(-0.758..."
411998,0.000000,10773,4840,1,ANN,0.670132,"[0.0014925788, 0.0037202446, 0.00070911075, 0....","[tensor(0.8572), tensor(-0.5422), tensor(-0.58..."
974130,-0.000109,22196,19731,0,ALS,0.941371,"[-0.0014244613, 0.0068824855, 0.030187089, -0....","[tensor(0.4234), tensor(-0.0738), tensor(-0.32..."
846631,-0.000149,15821,10015,0,ALS,1.023214,"[-0.0014244608, 0.00688246, 0.030187048, -0.02...","[tensor(0.9688), tensor(-0.5071), tensor(-0.56..."
...,...,...,...,...,...,...,...,...
289703,0.181800,28970,17472,1,ALS,1.067633,"[0.0024997697, 0.038088396, 0.0023099834, 0.03...","[tensor(0.3799), tensor(0.1506), tensor(0.1648..."
1079273,0.000007,27453,10017,0,ALS,0.922228,"[0.048556637, 0.046632916, 0.05640139, 0.00484...","[tensor(0.9688), tensor(-0.5071), tensor(-0.56..."
1102361,-0.000018,28608,1131,0,ALS,0.885037,"[0.048556607, 0.046632905, 0.056401387, 0.0048...","[tensor(0.1140), tensor(-0.5421), tensor(-1.23..."
1067715,-0.000783,26876,891,0,ALS,1.005122,"[-0.00142446, 0.006882466, 0.030187085, -0.021...","[tensor(0.9297), tensor(-0.4357), tensor(-0.29..."


In [124]:
d_len = 1149675
split = int(d_len*0.9)
ranking_df = ranking_df.sort_values('user')
data = ranking_df[['als_weight', 'ann_weight']]
labels = ranking_df['label']

train_data = data.iloc[0:split]
test_data = data.iloc[split:]

train_labels = labels.iloc[0:split]
test_labels = labels.iloc[split:]

In [125]:
train_labels

5          1
344536     1
344531     1
309753     1
530214     0
          ..
509610     1
278275     1
1086747    0
1086752    0
1086746    0
Name: label, Length: 1034707, dtype: int64

In [126]:
train_pool = Pool(train_data, train_labels, group_id=ranking_df.iloc[0:split]['user'])
test_pool = Pool(test_data, test_labels, group_id=ranking_df.iloc[split:]['user'])


In [127]:
ranker = CatBoostRanker(**{
    'loss_function': 'YetiRank',
    'learning_rate': 0.1,
    'early_stopping_rounds': 30,
    'custom_metric': ['RecallAt:top=1', 'RecallAt:top=10', 'PrecisionAt:top=10'],
    'eval_metric': 'PRAUC',
    'iterations': 10**4,
})

In [128]:
ranker.fit(
    train_pool,
    eval_set=test_pool,
    logging_level='Silent',
    plot=True,
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [129]:
ranker.save_model('./models/rank/model.catboost')

In [130]:
with open('./models/rank/dataset.pickle', 'wb') as file:
    pickle.dump(ranking_df, file)

In [131]:
ranker.best_score_

{'learn': {'RecallAt:top=1': 0.06143152980925358,
  'PrecisionAt:top=10': 0.9971575391691828,
  'RecallAt:top=10': 0.6128295904059067,
  'PRAUC': 0.9866046031862729},
 'validation': {'PFound': 0.9998570066730218,
  'RecallAt:top=1': 0.06317234470856811,
  'PrecisionAt:top=10': 0.9974261201143955,
  'RecallAt:top=10': 0.6305869149387051,
  'PRAUC': 0.9834544318868201}}

In [149]:
ranker.predict(test_data.iloc[4])

-1.2071630233745145